In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/datasets/tesla.csv')
data = data.dropna()

In [4]:
# Create features and labels
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

<ipython-input-4-6078241aa6e9>:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Date'] = pd.to_datetime(data['Date'])


In [5]:
# Create lagged features
for lag in range(1, 11):  # Increase the number of lagged features
    data[f'Close_lag_{lag}'] = data['Close'].shift(lag)

In [6]:
# Add more technical indicators as features
data['MA10'] = data['Close'].rolling(window=10).mean()
data['MA50'] = data['Close'].rolling(window=50).mean()
data['RSI'] = data['Close'].diff().rolling(window=14).apply(lambda x: np.mean(np.where(x > 0, x, 0)) / (np.mean(np.abs(x)) + 1e-10), raw=True)
data['EMA10'] = data['Close'].ewm(span=10, adjust=False).mean()
data['EMA50'] = data['Close'].ewm(span=50, adjust=False).mean()

In [7]:
data.dropna(inplace=True)

In [8]:
# Prepare feature and target arrays
features = ['Open', 'High', 'Low', 'Adj Close', 'Volume', 'MA10', 'MA50', 'RSI', 'EMA10', 'EMA50'] + [f'Close_lag_{lag}' for lag in range(1, 11)]
X = data[features]
y = data['Close']

In [9]:
# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [11]:
# Initialize the SVR model
svr = SVR()

In [12]:
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.2],
    'kernel': ['linear', 'poly', 'rbf']
}

In [13]:
# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100], 'epsilon': [0.01, 0.1, 0.2],
                         'kernel': ['linear', 'poly', 'rbf']},
             scoring='neg_mean_squared_error')

In [14]:
# Get the best estimator
best_svr = grid_search.best_estimator_

In [15]:
# Make predictions on the testing set
y_pred = best_svr.predict(X_test)

In [16]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [17]:
# Calculate a custom accuracy metric
def custom_accuracy(y_true, y_pred, tolerance=0.05):
    return np.mean(np.abs((y_true - y_pred) / y_true) <= tolerance)

accuracy = custom_accuracy(y_test, y_pred)

In [18]:
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2): {r2}')
print(f'Custom Accuracy: {accuracy * 100:.2f}%')

Mean Absolute Error (MAE): 0.003357291662016104
Mean Squared Error (MSE): 1.770263048197583e-05
R-squared (R2): 0.9999999986679201
Custom Accuracy: 100.00%


In [19]:
# Make a prediction on new data
new_data = pd.DataFrame({
    'Open': [300],
    'High': [310],
    'Low': [295],
    'Adj Close': [305],
    'Volume': [1000000],
    'MA10': [305],
    'MA50': [310],
    'RSI': [0.01],
    'EMA10': [305],
    'EMA50': [310],
    'Close_lag_1': [298],
    'Close_lag_2': [299],
    'Close_lag_3': [297],
    'Close_lag_4': [296],
    'Close_lag_5': [295],
    'Close_lag_6': [294],
    'Close_lag_7': [293],
    'Close_lag_8': [292],
    'Close_lag_9': [291],
    'Close_lag_10': [290]
})

In [20]:
new_data_scaled = scaler.transform(new_data)
new_prediction = best_svr.predict(new_data_scaled)
print(f'Predicted Close Price: {new_prediction[0]}')

Predicted Close Price: 305.82911006467026
